In [ ]:
%matplotlib inline
import os
import time
from matplotlib.pyplot import subplots
from pyFAI.gui import jupyter
import pyFAI
import fabio
from pyFAI.test.utilstest import UtilsTest
from pyFAI.calibrant import CALIBRANT_FACTORY
from pyFAI.goniometer import SingleGeometry
print(f"Using pyFAI version: {pyFAI.version}")
start_time = time.perf_counter()

# read folder

In [ ]:
folder_path = r'C:Users\b_tassone\Desktop\Anjani\Python\Data_Reduction\Convert_and_calibrate'

# Loop through all files in the folder

filename = 'Lab6_output_image.cbf'
# Open the cbs file and read the image data
frame = fabio.open(filename).data
# Do something with the image data here

# and now display the image
ax = jupyter.display(frame)
print (filename)

In [ ]:
# This allow to measure approximatively the position of the beam center ...
x = 600 # x-coordinate of the beam-center in pixels
y = 100 # y-coordinate of the beam-center in pixels
d = 150 # This is the distance in mm (unit used by Fit2d)
wl = 0.826565e-10 # Equivalent to 15 keV 
tilt = 25 

In [ ]:
import pyFAI

# Definition of the detector and of the calibrant:
pilatus = pyFAI.detector_factory("Pilatus100k")
calibrant = pyFAI.calibrant.CALIBRANT_FACTORY("LaB6")
calibrant.wavelength = wl  # You need to define 'wl' before using it


In [ ]:
# Set the guessed geometry
initial = pyFAI.geometry.Geometry(detector=pilatus, wavelength=wl)
initial.setFit2D(d,x,y,tilt)
initial

In [ ]:
# The SingleGeometry object (from goniometer) allows to extract automatically ring and calibrate
sg = SingleGeometry(filename, frame, calibrant=calibrant, detector=pilatus, geometry=initial)
sg.extract_cp(max_rings=1)

In [ ]:
#Control point and rings do not overlap well initially (this was a guessed geometry)
ax = jupyter.display(sg=sg)

In [ ]:
# Refine the geometry ... here in SAXS geometry, the rotation is fixed in orthogonal setup
sg.geometry_refinement.refine2(fix=["rot1", "rot2", "rot3", "wavelength"])
sg.get_ai()

In [ ]:
ax = jupyter.display(sg=sg)

In [ ]:
#Save the geometry obtained
sg.geometry_refinement.save("Autoxidation_august_2023.poni")
with open("Autoxidation_august_2023.poni") as f:
    print(f.read())

In [ ]:
#Use the geometry to perform an azimuthal integration
ai = sg.get_ai()
res = ai.integrate1d(frame, 1000)
ax = jupyter.plot1d(res,calibrant=behenate)
#ax = jupyter.plot1d(res, calibrant=behenate) 
ax.set_xlim([0.5, 3])
ax.set_ylim([0, 200])
#plt.semilogy(res.radial, res.intensity, label=filename)